<a href="https://colab.research.google.com/github/nmcardoso/splus-model/blob/master/splus_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x 
import random
import tensorflow as tf
import numpy as np
from keras import backend as K
import os
import glob
import multiprocessing as mp
import shutil

from keras import layers
from keras import models
from keras import optimizers
from keras import utils
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications import VGG16

import matplotlib.pyplot as plt

from progressbar import progressbar

!pip3 install git+https://github.com/nmcardoso/fitsbook-python
import fitsbook as fb

In [2]:
# Reprodutibilidade
# https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development

os.environ['PYTHONHASHSEED'] = '12'
np.random.seed(123)
random.seed(1234)
tf.set_random_seed(12345)

# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, 
#                               inter_op_parallelism_threads=1)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)

print('Sementes Plantadas!')

Sementes Plantadas!


In [0]:
!unzip /content/drive/'My Drive'/datasets/stamps_scaled.zip -d dataset

In [0]:
!tar -zxvf /content/drive/'My Drive'/datasets/dataset.tar.gz

In [0]:
class NpySequence(utils.Sequence):
  """Generate data for Keras"""
  def __init__(self, path, image_size=(256, 256), channels=3, batch_size=32, shuffle=True):
    self.path = path
    self.image_size = image_size
    self.channels = channels
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.files = glob.glob(os.path.join(self.path, '*/*.npy'))
    np.random.shuffle(self.files)
    self.labels_map = self.__get_labels_map()
    # self.on_epoch_end()
  
  def __len__(self):
    """The number of batches per epoch"""
    return int(np.floor(len(self.files) / self.batch_size))
  
  def __getitem__(self, index):
    """Generate one batch of data"""
    partition = self.files[index * self.batch_size : (index + 1) * self.batch_size]
    X, Y = self.__data_generation(partition)
    return X, Y
  
  def on_epoch_end(self):
    """Shuffle files list and remap labels"""
    if self.shuffle:
      np.random.shuffle(self.files)
    self.labels_map = self.__get_labels_map()
    
  def __get_labels_map(self):
    """Create labels dict from dir names"""
    labels = [name for name in os.listdir(self.path) if os.path.isdir(os.path.join(self.path, name)) and name[0] != '.']
    labels.sort()
    return {l:i for i,l in enumerate(labels)}

  def __data_generation(self, partition):
    """Load npy file and associate with a label"""
    X = np.empty((self.batch_size, *self.image_size, self.channels))
    Y = np.empty((self.batch_size,), dtype=int)

    for i, file_path in enumerate(partition):
      X[i,] = np.load(file_path)
      Y[i] = self.labels_map[file_path.split('/')[-2]]

    return X, Y

In [0]:
def preprocess():
  global train_generator, test_generator, val_generator
  train_generator = NpySequence('/content/dataset/train')
  val_generator = NpySequence('/content/dataset/val')

def compile_model(image_size):
  global model
  model = models.Sequential()
  model.name = 'sdss_convolutional'
  model.add(layers.Conv2D(256, (3, 3), activation='relu', input_shape=(image_size + (3,))))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer=optimizers.RMSprop(lr=5e-5),
                loss='binary_crossentropy',
                metrics=['acc'])

def fit(epochs=1):
  global model, history, train_generator, val_generator
  history = model.fit_generator(
      train_generator,
      steps_per_epoch=80,
      validation_steps=30,
      epochs=epochs,
      validation_data=val_generator,
      callbacks=[fb.callbacks.FitsbookCallback()],
      use_multiprocessing=True,
      workers=4
  )

In [0]:
preprocess()
compile_model((256, 256))
fit(300)

[Fitsbook]: Monitoring this training in real time https://natan.ninja/#/stats/31
Epoch 1/300
80/80 [==============================] - 159s 2s/step - loss: 0.5851 - acc: 0.7684 - val_loss: 0.5576 - val_acc: 0.7594
Epoch 2/300
80/80 [==============================] - 137s 2s/step - loss: 0.5697 - acc: 0.7539 - val_loss: 0.5507 - val_acc: 0.7594
Epoch 3/300
80/80 [==============================] - 152s 2s/step - loss: 0.5433 - acc: 0.7707 - val_loss: 0.5275 - val_acc: 0.7792
Epoch 4/300
80/80 [==============================] - 146s 2s/step - loss: 0.5462 - acc: 0.7695 - val_loss: 0.5527 - val_acc: 0.7542
Epoch 5/300
80/80 [==============================] - 138s 2s/step - loss: 0.5453 - acc: 0.7648 - val_loss: 0.5299 - val_acc: 0.7750
Epoch 6/300
80/80 [==============================] - 151s 2s/step - loss: 0.5290 - acc: 0.7680 - val_loss: 0.5123 - val_acc: 0.7802
Epoch 7/300
80/80 [==============================] - 142s 2s/step - loss: 0.5150 - acc: 0.7785 - val_loss: 0.5473 - val_acc: 0.